In [46]:
import subprocess
import json
from dotenv import load_dotenv
from constants import *
import os

# Load and set environment variables
load_dotenv()
mnemonic=os.getenv("mnemonic")

In [47]:
def derive_wallets(mnemonic,coin,format):
    command =f'php ./derive -g --mnemonic="{mnemonic}" --cols=all --coin={coin} --numderive=3 --format={format}'

    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)


In [48]:
coins = {
    ETH: derive_wallets(mnemonic,ETH,'json'),
    BTCTEST: derive_wallets(mnemonic,BTCTEST,'json')
}
coins

{'eth': [{'path': "m/44'/60'/0'/0/0",
   'address': '0xB15cd61B5C9aF5ca1DD42f9AbBD7E3AA99659d05',
   'xprv': 'xprvA3Q44VyxZiU6k5PQjQj2YJVAbSFwcNMAY76oH2mPJ1aD3zSRtSsQGzpsEhevdheL97aM5sPhsT1tfFasTBcjUi92RRvcMqKnXJHM6PweuyE',
   'xpub': 'xpub6GPQU1WrQ62PxZTsqSG2uSRu9U6S1q51uL2Q5RAzrM7BvnmaRzBepo9M5xBwDJdpqbS34reZt3LepZ9SbT3JRmJCJuCaqj3Xw5mvwFiX71C',
   'privkey': '0xb38a98016af2cae94437be96f935666b09421cfde321600eb398cdd330f9ec03',
   'pubkey': '02723f2db8f9625c17e97413e6ea1392c144d2ee8c6539e680c425eb2c9a968e40',
   'pubkeyhash': 'a43fff8443472390d82ea6362fbfb3636e96fcd0',
   'index': 0},
  {'path': "m/44'/60'/0'/0/1",
   'address': '0x3751721bb02eeAb0D40931987ABC3c14C8DA2a65',
   'xprv': 'xprvA3Q44VyxZiU6p8mMpguYNxFEL8qLnrwwXW2Z6rMrDYyNLeNKTy8WLaPfZfXmCny35zVA1Znru6aGMxT1iN236bV1V1fj1UZF3qCr9Z8F4f3',
   'xpub': 'xpub6GPQU1WrQ62Q2cqpviSYk6BxtAfqCKfntix9uEmTmtWMDShU1WSktNi9QwZA8oAA4bysHSLJ7NCBqEYSMebvStebj9DdxkFisFHxpnYJEJQ',
   'privkey': '0x64ddf7fbcc1f4dcb8b9f7ba095ec52f2de9ea54ec151a7

In [38]:
def priv_key_to_account(coin, priv_key):
    if coin == 'BTCTEST':
        return bit.PrivateKeyTestnet(priv_key) 
    elif coin == 'ETH':
        return Account.privateKeyToAccount(priv_key)

In [40]:
def create_tx(coin, account, to, amount):
    if coin == ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, 
             "to": to, 
             "value": amount
            })
        return {"from": account.address, 
                "to": to, 
                "value": amount, 
                "gasPrice": w3.eth.gasPrice, 
                "gas": gasEstimate, 
                "nonce": w3.eth.getTransactionCount(account.address), 
                "chainID": w3.eth.chain_id}
    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTCTEST)])

In [41]:
def send_tx(coin, account, to, amount):
    if coin == ETH:
        raw_tx = create_tx(coin, account, to, amount)
        signed = account.signTransaction(raw_tx)
        return w3.eth.sendRawTransaction(signed.rawTransaction)
    if coin == BTCTEST:
        raw_tx = create_tx(coin, account, to, amount)
        signed = account.sign_transaction(raw_tx)
        return NetworkAPI.broadcast_tx_testnet(signed)